In [15]:
from sklearn import svm
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, make_scorer
from hyperopt import rand, tpe, fmin, hp, Trials
import time
from pyswarm import pso
from simanneal import Annealer
import cma

In [16]:
xtreino = np.load("Xtreino5.npy")
ytreino = np.load("ytreino5.npy")
xtest = np.load("Xteste5.npy")
ytest = np.load("yteste5.npy")

clf = svm.SVR(kernel="rbf")

<hr>

<h2>Intervalo dos hiperparâmetros:</h2><br>

<ul>
    <li>C entre $2^{-5}$ e $2^{15}$ (uniforme nos expoentes)</li>
    <li>gamma entre $2^{-15}$ e $2^3$ (uniforme nos expoentes)</li>
    <li>epsilon entre 0.05 a 1.0 (uniforme neste intervalo)</li>
</ul>

<hr>

<h3>1 - Random search</h3>

In [28]:
n_iters = 125
param_distributions = {"C":np.random.uniform(2**-5,2**15,n_iters),
                       "gamma":np.random.uniform(2**-15,2**3,n_iters),
                       "epsilon":np.random.uniform(10**-2,1,n_iters)}

score = make_scorer(mean_squared_error)

svr = RandomizedSearchCV(clf,param_distributions , n_iter=n_iters, scoring=score, cv=5, iid=True)
svr.fit(xtreino, ytreino)
error = mean_squared_error(svr.predict(xtest), ytest)

In [29]:
print("Parameters: {0}".format(svr.cv_results_['params'][0]))
print("Mean squared error: %f" % (error))

Parameters: {'gamma': 2.465835903429985, 'epsilon': 0.4384722419842344, 'C': 3097.5557028954627}
Mean squared error: 67.141073


<hr>
<br>
<h3>2 - Grid search</h3>
<hr>

In [30]:
n_iters = 5

param_distributions = {"C":np.random.uniform(2**-5,2**15,n_iters),
                       "gamma":np.random.uniform(2**-15,2**3,n_iters),
                       "epsilon":np.random.uniform(10**-2,1,n_iters)}


score = make_scorer(mean_squared_error)

svr =  GridSearchCV(clf,param_distributions, scoring=score, iid=True, cv=5)

svr.fit(xtreino, ytreino)
error = mean_squared_error(svr.predict(xtest), ytest)


In [31]:
print("Parameters: {0}".format(svr.cv_results_['params'][0]))
print("Mean squared error: %f" % (error))

Parameters: {'C': 11824.737465935685, 'epsilon': 0.6192314771982733, 'gamma': 6.890367090185687}
Mean squared error: 67.144007


<hr>
<br>
<h3>3 - Otimização bayesiana</h3>
<hr>

In [32]:
def objective(args):
    gamma = args[0][1]
    C = args[1][1]
    epsilon = args[2][1]
    clf = svm.SVR(kernel="rbf",gamma=gamma,C=C,epsilon=epsilon)
    clf.fit(xtreino,ytreino)
    return mean_squared_error(svr.predict(xtest), ytest)

space = [('gamma', hp.uniform('gamma', 2**-15,2**3)),
        ('C', hp.uniform('C', 2**-5,2**15)),
        ('epsilon', hp.uniform('epsilon', 10**-2,1))]

In [33]:
tpe_algo = tpe.suggest

tpe_trials = Trials()
tpe_best = fmin(fn=objective, space=space, algo=tpe_algo, trials=tpe_trials, 
                max_evals=125, rstate= np.random.RandomState(50))




100%|██████████| 125/125 [00:01<00:00, 63.99it/s, best loss: 67.14400667426125]


In [34]:
svr = SVR(kernel="rbf",C=tpe_best["C"],epsilon = tpe_best["epsilon"],gamma = tpe_best["gamma"])
svr.fit(xtreino, ytreino)
error = mean_squared_error(svr.predict(xtest), ytest)

print("Parameters: {'C': %f, 'epsilon': %f, 'gamma': %f}" % ((tpe_best["C"]),(tpe_best["epsilon"]),tpe_best["gamma"]))
print("Mean squared error: %f" % (error))

Parameters: {'C': 19612.051235, 'epsilon': 0.588700, 'gamma': 7.969371}
Mean squared error: 67.123088


<hr>
<br>
<h3>4 - PSO</h3>
<hr>

In [39]:
def run_SVR(c,gamma,epsilon):
    #Run SVM
    clf = SVR(gamma=gamma, C=c, epsilon=epsilon,kernel='rbf')
    clf.fit(xtreino,ytreino)
    Y_pred = clf.predict(xtest)
    # calulate MSE
    mse = np.sum((Y_pred - ytest)**2)
    mse /= Y_pred.shape[0]
    return mse
def run_SVR_params(params):
    c = 2**params['c']
    gamma = 2**params['gamma']
    epsilon = params['epsilon']
    mse = run_SVR(c,gamma,epsilon)
    return mse

def run_SVR_vect(X):
    c = 2**X[0]
    gamma = 2**X[1]
    epsilon = X[2]
    mse = run_SVR(c,gamma,epsilon)
    return mse

#Search space
lb = [-5, -15, 0.05]
ub = [15,3,1]

xopt, fopt = pso(run_SVR_vect, lb, ub, swarmsize=11, omega=0.5, phip=0.5, phig=0.5, minfunc=1e-8, maxiter=11, debug=False)

Stopping search: maximum iterations reached --> 11


In [40]:
pso_clf = SVR(C=2**(xopt[0]),gamma=2**(xopt[1]),epsilon=xopt[2])
pso_clf.fit(xtreino, ytreino)
y_predito = pso_clf.predict(xtest)
error = mean_squared_error(ytest, y_predito)

In [43]:
print("Parameters: {'C': %f, 'epsilon': %f, 'gamma': %f}" % (2**(xopt[0]),xopt[2],2**(xopt[1])))
print("Mean squared error: %f" % (error))

Parameters: {'C': 27264.048703, 'epsilon': 0.276275, 'gamma': 0.000031}
Mean squared error: 11.665856


<hr>
<br>
<h3>5 - Simulated Annealing</h3>
<hr>

In [44]:
def objective(args):
    tam_treino=xtreino.shape[0]
    perm=np.random.permutation(np.arange(tam_treino))
    idx_treino=perm[:int(tam_treino*0.85)]
    idx_val=perm[int(tam_treino*0.85):]
    svm = SVR(kernel='rbf',C=2**args['C'], gamma=2**args['gamma'], epsilon=args['epsilon'])
    svm.fit(xtreino[idx_treino], ytreino[idx_treino])
    return mean_squared_error(ytreino[idx_val], svm.predict(xtreino[idx_val]))

class OptSVR(Annealer):
    def __init__(self, state):
        super(OptSVR,self).__init__(state)
    def move(self):
        new_C = np.random.uniform(low=-5, high=15)
        new_gamma = np.random.uniform(low=-15, high=3)
        new_epsilon = np.random.uniform(low=0.05, high=1.0)
        self.state['C'] = new_C
        self.state['gamma'] = new_gamma
        self.state['epsilon'] = new_epsilon
    def energy(self):
        return objective(self.state)
    
init_C = np.random.uniform(low=-5, high=15)
init_gamma = np.random.uniform(low=-15, high=3)
init_epsilon = np.random.uniform(low=0.05, high=1.0)

init_state={'C': init_C,'gamma': init_gamma,'epsilon': init_epsilon}
svrpp = OptSVR(init_state)
svrpp.steps = 125

svrpp.copy_strategy = "deepcopy"
state, e = svrpp.anneal()
anneal_clf = SVR(C=2**(state['C']), gamma=2**(state['gamma']), epsilon=state['epsilon'])

anneal_clf.fit(xtreino, ytreino)
anneal_y_predito=anneal_clf.predict(xtest)
error = mean_squared_error(anneal_y_predito, ytest)
state['C']=2**(state['C'])
state['gamma']=2**(state['gamma'])



 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     2.50000          7.62     0.00%     0.00%     0:00:07     0:00:00

In [45]:
print("Parameters: {'C': %f, 'epsilon': %f, 'gamma': %f}" % (state['C'], state['gamma'], state['epsilon']))
print("Mean squared error: %f" % (error))

Parameters: {'C': 5838.440009, 'epsilon': 0.000032, 'gamma': 0.674824}
Mean squared error: 14.413705


<hr>
<br>
<h3>6 - CMA-ES</h3>
<hr>

In [47]:
def objective_cmaes(args):
    tam_treino=xtreino.shape[0]
    perm=np.random.permutation(np.arange(tam_treino))
    idx_treino=perm[:int(tam_treino*0.85)]
    idx_val=perm[int(tam_treino*0.85):]
    if (args[2] < 0):
        args[2] = 0
    clf = svm.SVR(kernel='rbf', C=2**args[0],gamma=2**args[1],epsilon=args[2])
    clf.fit(xtreino[idx_treino], ytreino[idx_treino])
    return mean_squared_error(ytreino[idx_val], clf.predict(xtreino[idx_val]))

init_C = np.random.uniform(low=-5, high=15)
init_gamma = np.random.uniform(low=-15, high=3)
init_epsilon = np.random.uniform(low=0.05, high=1.0)
init_state = [init_C, init_gamma, init_epsilon]
es = cma.CMAEvolutionStrategy(init_state,0.3,{'maxfevals': 125,"CMA_stds": [20, 20, 1]}).optimize(objective_cmaes).result

(3_w,7)-aCMA-ES (mu_w=2.3,w_1=58%) in dimension 3 (seed=918622, Thu May  2 20:17:43 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      7 4.274083051234745e+01 1.0e+00 2.78e-01  2e-01  6e+00 0:00.1
    2     14 4.630522374453901e+01 1.2e+00 3.94e-01  4e-01  8e+00 0:00.3
    3     21 2.614134150329174e+01 1.1e+00 3.42e-01  4e-01  7e+00 0:06.4
   10     70 2.145251981984535e+01 2.3e+00 2.26e-01  2e-01  3e+00 0:11.5
   13     91 1.030061239195111e+01 2.7e+00 2.00e-01  3e-01  2e+00 0:16.4
   16    112 1.031134973992588e+01 3.4e+00 1.55e-01  2e-01  1e+00 0:23.7
   18    126 1.196600049968051e+01 3.5e+00 1.44e-01  2e-01  1e+00 0:28.8


In [51]:
xbest = es.xbest
cmaes_clf = svm.SVR(C=2**(xbest[0]),
gamma=2**(xbest[1]),
epsilon=xbest[2])
cmaes_clf.fit(xtreino, ytreino)
cmaes_y_predito=cmaes_clf.predict(xtest)
error = mean_squared_error(cmaes_y_predito, ytest)

parameters = {'C': 2**(xbest[0]),'gamma': 2**(xbest[1]),'epsilon': xbest[2]}

print("Parameters:",parameters)
print("Mean squared error: %f" % (error))

Parameters: {'C': 21115.262788784814, 'gamma': 4.881109839331006e-06, 'epsilon': 0.4487052589763859}
Mean squared error: 17.564983
